# Spark Preparation
We check if we are in Google Colab.  If this is the case, install all necessary packages.

To run spark in Colab, we need to first install all the dependencies in Colab environment i.e. Apache Spark 3.3.2 with hadoop 3.3, Java 8 and Findspark to locate the spark in the system. The tools installation can be carried out inside the Jupyter Notebook of the Colab.
Learn more from [A Must-Read Guide on How to Work with PySpark on Google Colab for Data Scientists!](https://www.analyticsvidhya.com/blog/2020/11/a-must-read-guide-on-how-to-work-with-pyspark-on-google-colab-for-data-scientists/)

In [55]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [56]:
if IN_COLAB:
    !apt-get install openjdk-8-jdk-headless -qq > /dev/null
    !wget -q https://dlcdn.apache.org/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
    !tar xf spark-3.5.1-bin-hadoop3.tgz
    !mv spark-3.5.1-bin-hadoop3 spark
    !pip install -q findspark
    import os
    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
    os.environ["SPARK_HOME"] = "/content/spark"

# Start a Local Cluster

In [57]:
import findspark
findspark.init()
spark_url = 'local'

from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master(spark_url)\
        .appName('Spark ML')\
        .getOrCreate()

# Spark Assignment

Based on the movie review dataset in 'netflix-rotten-tomatoes-metacritic-imdb.csv', answer the below questions.

**Note:** do not clean or remove missing data

In [58]:
import urllib.request

url = "https://raw.githubusercontent.com/pvateekul/2110446_DSDE_2023s2/main/code/Week10_Spark/netflix-rotten-tomatoes-metacritic-imdb.csv"
filename = "netflix-rotten-tomatoes-metacritic-imdb.csv"

# Download the file from the URL
urllib.request.urlretrieve(url, filename)
df = spark.read.option("delimiter", ",").option("header", True).csv(filename)

In [59]:
df.show(10)

+-------------------+--------------------+--------------------+--------------------+---------------+----------------+--------------------+------------+---------------+--------------------+--------------------+-----------+----------+---------------------+----------------+---------------+--------------------+----------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+------------+
|              Title|               Genre|                Tags|           Languages|Series or Movie|Hidden Gem Score|Country Availability|     Runtime|       Director|              Writer|              Actors|View Rating|IMDb Score|Rotten Tomatoes Score|Metacritic Score|Awards Received|Awards Nominated For| Boxoffice|Release Date|Netflix Release Date|    Production House|        Netflix Link|           IMDb Link|             Summary|IMDb Votes|               Image|      

In [60]:
df.columns

['Title',
 'Genre',
 'Tags',
 'Languages',
 'Series or Movie',
 'Hidden Gem Score',
 'Country Availability',
 'Runtime',
 'Director',
 'Writer',
 'Actors',
 'View Rating',
 'IMDb Score',
 'Rotten Tomatoes Score',
 'Metacritic Score',
 'Awards Received',
 'Awards Nominated For',
 'Boxoffice',
 'Release Date',
 'Netflix Release Date',
 'Production House',
 'Netflix Link',
 'IMDb Link',
 'Summary',
 'IMDb Votes',
 'Image',
 'Poster',
 'TMDb Trailer',
 'Trailer Site']

In [61]:
cols = [c.replace(' ','_') for c in df.columns]
df = df.toDF(*cols)
df.columns

['Title',
 'Genre',
 'Tags',
 'Languages',
 'Series_or_Movie',
 'Hidden_Gem_Score',
 'Country_Availability',
 'Runtime',
 'Director',
 'Writer',
 'Actors',
 'View_Rating',
 'IMDb_Score',
 'Rotten_Tomatoes_Score',
 'Metacritic_Score',
 'Awards_Received',
 'Awards_Nominated_For',
 'Boxoffice',
 'Release_Date',
 'Netflix_Release_Date',
 'Production_House',
 'Netflix_Link',
 'IMDb_Link',
 'Summary',
 'IMDb_Votes',
 'Image',
 'Poster',
 'TMDb_Trailer',
 'Trailer_Site']

In [62]:
from pyspark.sql.functions import col
cols = ['Awards_Received', 'Awards_Nominated_For', 'IMDb_Votes']
for c in cols:
    df = df.withColumn(c, col(c).cast('int'))
cols = ['Hidden_Gem_Score', 'IMDb_Score', 'Rotten_Tomatoes_Score', 'Metacritic_Score']
for c in cols:
    df = df.withColumn(c, col(c).cast('double'))

In [63]:
df.printSchema()

root
 |-- Title: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Tags: string (nullable = true)
 |-- Languages: string (nullable = true)
 |-- Series_or_Movie: string (nullable = true)
 |-- Hidden_Gem_Score: double (nullable = true)
 |-- Country_Availability: string (nullable = true)
 |-- Runtime: string (nullable = true)
 |-- Director: string (nullable = true)
 |-- Writer: string (nullable = true)
 |-- Actors: string (nullable = true)
 |-- View_Rating: string (nullable = true)
 |-- IMDb_Score: double (nullable = true)
 |-- Rotten_Tomatoes_Score: double (nullable = true)
 |-- Metacritic_Score: double (nullable = true)
 |-- Awards_Received: integer (nullable = true)
 |-- Awards_Nominated_For: integer (nullable = true)
 |-- Boxoffice: string (nullable = true)
 |-- Release_Date: string (nullable = true)
 |-- Netflix_Release_Date: string (nullable = true)
 |-- Production_House: string (nullable = true)
 |-- Netflix_Link: string (nullable = true)
 |-- IMDb_Link: string (nu

## What is the maximum and average of the overall hidden gem score?

In [64]:
from pyspark.sql.functions import avg, max
df.select(avg('Hidden_Gem_Score'), max('Hidden_Gem_Score')).show()

+---------------------+---------------------+
|avg(Hidden_Gem_Score)|max(Hidden_Gem_Score)|
+---------------------+---------------------+
|    5.937551386501226|                  9.8|
+---------------------+---------------------+



## How many movies that are available in Korea?

In [65]:
rows = df.collect()
row_5_country_availability = rows[4]['Country_Availability']
print(row_5_country_availability)

Lithuania,Poland,France,Iceland,Italy,Spain,Greece,Czech Republic,Belgium,Portugal,Canada,Hungary,Mexico,Slovakia,Sweden,South Africa,Netherlands,Germany,Thailand,Turkey,Singapore,Romania,Argentina,Israel,Switzerland,Australia,United Kingdom,Brazil,Malaysia,India,Colombia,Hong Kong,Japan,South Korea,United States,Russia


In [66]:
from pyspark.sql.functions import col

df_korea = df.filter(col('Country_Availability').contains('Korea'))
count_korea = df_korea.count()
print(count_korea)
df_korea.select('Country_Availability').show(10)

4845
+--------------------+
|Country_Availability|
+--------------------+
|Lithuania,Poland,...|
|         South Korea|
|         South Korea|
|Lithuania,Poland,...|
|         South Korea|
|         South Korea|
|         South Korea|
|         South Korea|
|         South Korea|
|         South Korea|
+--------------------+
only showing top 10 rows



## Which director has the highest average hidden gem score?

In [67]:
df_director_score = df.groupby('Director').agg({'Hidden_Gem_Score': 'avg'})

max_score = df_director_score.agg(max('avg(Hidden_Gem_Score)')).collect()[0][0]
df_max_director = df_director_score.where(df_director_score['avg(Hidden_Gem_Score)'] == max_score)

df_max_director.show()

+-----------+---------------------+
|   Director|avg(Hidden_Gem_Score)|
+-----------+---------------------+
|Dorin Marcu|                  9.8|
+-----------+---------------------+



In [69]:
# Filter the DataFrame for rows where the 'Director' is 'Dorin Marcu'
df_director = df.filter(df['Director'] == 'Dorin Marcu')
df_score = df_director.select('Hidden_Gem_Score')
df_score.show()

+----------------+
|Hidden_Gem_Score|
+----------------+
|             9.8|
|             9.8|
+----------------+



## How many genres are there in the dataset?

In [68]:
from pyspark.sql.functions import split, explode

df = df.withColumn('Genre', split(df['Genre'], ', '))
df = df.withColumn('Genre', explode(df['Genre']))
distinct_genre_count = df.select('Genre').distinct().count()
print(distinct_genre_count)
df.select('Genre').distinct().show()

28
+-----------+
|      Genre|
+-----------+
|      Crime|
|    Romance|
|   Thriller|
|  Adventure|
|      Drama|
|        War|
|Documentary|
| Reality-TV|
|     Family|
|    Fantasy|
|  Game-Show|
|      Adult|
|    History|
|    Mystery|
|    Musical|
|  Animation|
|      Music|
|  Film-Noir|
|     Horror|
|      Short|
+-----------+
only showing top 20 rows

